In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
data_dir = '/content/drive/MyDrive/main'
batch_size = 32
img_height = 180
img_width = 180

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 17676 files belonging to 7 classes.
Using 14141 files for training.


In [4]:
train_ds

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 17676 files belonging to 7 classes.
Using 3535 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['0', '1', '2', '3', '4', '5', '6']


In [7]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [8]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [9]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.6561441


In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
num_classes = 7

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(32, activation='relu'),
  layers.Dense(num_classes)
])

In [13]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [14]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

Epoch 1/20
442/442 [==============================] - 8865s 20s/step - loss: 1.8781 - accuracy: 0.2015 - val_loss: 1.1416 - val_accuracy: 0.5689
Epoch 2/20
442/442 [==============================] - 10s 22ms/step - loss: 1.1035 - accuracy: 0.5706 - val_loss: 0.7746 - val_accuracy: 0.6987
Epoch 3/20
442/442 [==============================] - 10s 22ms/step - loss: 0.7647 - accuracy: 0.6941 - val_loss: 0.6185 - val_accuracy: 0.7740
Epoch 4/20
442/442 [==============================] - 10s 22ms/step - loss: 0.5902 - accuracy: 0.7731 - val_loss: 0.4937 - val_accuracy: 0.8238
Epoch 5/20
442/442 [==============================] - 10s 22ms/step - loss: 0.4355 - accuracy: 0.8380 - val_loss: 0.4126 - val_accuracy: 0.8467
Epoch 6/20
442/442 [==============================] - 10s 22ms/step - loss: 0.3354 - accuracy: 0.8765 - val_loss: 0.3599 - val_accuracy: 0.8736
Epoch 7/20
442/442 [==============================] - 10s 22ms/step - loss: 0.2569 - accuracy: 0.9079 - val_loss: 0.3313 - val_accuracy

In [15]:
model.save('drive/MyDrive/Model/2', save_format='h5')

In [16]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0686 - accuracy: 0.9765 - val_loss: 0.2136 - val_accuracy: 0.9539
Epoch 2/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0651 - accuracy: 0.9780 - val_loss: 0.2003 - val_accuracy: 0.9598
Epoch 3/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0690 - accuracy: 0.9782 - val_loss: 0.2106 - val_accuracy: 0.9528
Epoch 4/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0532 - accuracy: 0.9813 - val_loss: 0.2540 - val_accuracy: 0.9468
Epoch 5/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0555 - accuracy: 0.9818 - val_loss: 0.2420 - val_accuracy: 0.9471


In [17]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0586 - accuracy: 0.9791 - val_loss: 0.2662 - val_accuracy: 0.9434
Epoch 2/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0481 - accuracy: 0.9822 - val_loss: 0.2538 - val_accuracy: 0.9525
Epoch 3/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0530 - accuracy: 0.9828 - val_loss: 0.2566 - val_accuracy: 0.9511
Epoch 4/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0605 - accuracy: 0.9803 - val_loss: 0.2233 - val_accuracy: 0.9559
Epoch 5/5
442/442 [==============================] - 10s 22ms/step - loss: 0.0498 - accuracy: 0.9827 - val_loss: 0.2000 - val_accuracy: 0.9579


In [18]:
model.save('drive/MyDrive/Model/3', save_format='h5')